<img src = "https://images2.imgbox.com/60/09/VFwl5LOq_o.jpg" width="400">

# 4. Window Functions
---

You will learn about window functions and how to pass aggregate functions along a dataset. You will also learn how to calculate running totals and partitioned averages.

In [ ]:
# %pip install ipython-sql

In [1]:
%load_ext sql

In [2]:
%sql sqlite:///data/soccer.db

'Connected: @data/soccer.db'

## The match is OVER
---

The `OVER()` clause allows you to pass an aggregate function down a data set, similar to subqueries in `SELECT`. The `OVER()` clause offers significant benefits over subqueries in select -- namely, your queries will run faster, and the `OVER()` clause has a wide range of additional functions and clauses you can include with it that we will cover later on in this chapter.

In this exercise, you will revise some queries from previous chapters using the `OVER()` clause.

### Instructions

Select the match ID, country name, season, home, and away goals from the `match` and `country` tables.

Complete the query that calculates the average number of goals scored overall and then includes the aggregate value in each row using a window function.

In [4]:
%%sql

SELECT m.id,
       c.NAME   AS country,
       m.season,
       m.home_goal,
       m.away_goal,
       AVG(m.home_goal + m.away_goal)
         OVER() AS overall_avg
FROM   match AS m
       LEFT JOIN country AS c
              ON m.country_id = c.id
LIMIT 20

 * sqlite:///data/soccer.db
Done.


id,country,season,home_goal,away_goal,overall_avg
997,Belgium,2012/2013,1,1,2.7293006271334446
998,Belgium,2012/2013,2,4,2.7293006271334446
999,Belgium,2012/2013,5,2,2.7293006271334446
1000,Belgium,2012/2013,0,1,2.7293006271334446
1001,Belgium,2012/2013,4,2,2.7293006271334446
1002,Belgium,2012/2013,3,1,2.7293006271334446
1003,Belgium,2012/2013,2,0,2.7293006271334446
1004,Belgium,2012/2013,3,3,2.7293006271334446
1005,Belgium,2012/2013,1,1,2.7293006271334446
1006,Belgium,2012/2013,2,1,2.7293006271334446


## What's OVER here?
---

Window functions allow you to create a `RANK` of information according to any variable you want to use to sort your data. When setting this up, you will need to specify what column/calculation you want to use to calculate your rank. This is done by including an `ORDER BY` clause inside the `OVER()` clause. Below is an example:

`SELECT id,
        RANK() OVER(ORDER BY home_goal) AS rank
 FROM   match`

In this exercise, you will create a data set of ranked matches according to which leagues, on average, score the most goals in a match.

### Instructions

Select the league name and average total goals scored from `league` and `match`.

Complete the window function so it calculates the rank of average goals scored across all leagues in the database.

Order the rank by the average total of home and away goals scored.

## Flip OVER your results
---

In the last exercise, the rank generated in your query was organized from smallest to largest. By adding `DESC` to your window function, you can create a rank sorted from largest to smallest.

`SELECT id,
         RANK() OVER(ORDER BY home_goal DESC) AS rank
 FROM    match`

### Instructions

Complete the same parts of the query as the previous exercise.

Complete the window function to rank each league from highest to lowest average goals scored.

Order the main query by the rank you just created.

In [7]:
%%sql

SELECT l.name AS league,
       AVG(m.home_goal + m.away_goal) AS avg_goals,
       RANK() OVER(ORDER BY AVG(m.home_goal + m.away_goal) DESC) AS league_rank
FROM league AS l
LEFT JOIN match AS m 
ON l.id = m.country_id
WHERE m.season = '2011/2012'
GROUP BY l.name
ORDER BY league_rank

 * sqlite:///data/soccer.db
Done.


league,avg_goals,league_rank
Netherlands Eredivisie,3.258169934640523,1
Germany 1. Bundesliga,2.8594771241830066,2
England Premier League,2.805263157894737,3
Spain LIGA BBVA,2.763157894736842,4
Portugal Liga ZON Sagres,2.6416666666666666,5
Scotland Premier League,2.6359649122807016,6
Switzerland Super League,2.623456790123457,7
Italy Serie A,2.583798882681564,8
France Ligue 1,2.5157894736842104,9
Poland Ekstraklasa,2.1958333333333333,10


## PARTITION BY a column
---

The `PARTITION BY` clause allows you to calculate separate "windows" based on columns you want to divide your results. For example, you can create a single column that calculates an overall average of goals scored for each season.

In this exercise, you will be creating a data set of games played by *Legia Warszawa* (Warsaw League), the top ranked team in Poland, and comparing their individual game performance to the overall average for that season.

Where do you see more outliers? Are they *Legia Warszawa's* home or away games?

### Instructions

Complete the two window functions that calculate the home and away goal averages. Partition the window functions by season to calculate separate averages for each season.

Filter the query to only include matches played by Legia Warszawa, `id = 8673`.

In [9]:
%%sql

SELECT date,
       season,
       home_goal,
       away_goal,
CASE WHEN hometeam_id = 8673 THEN 'home' 
    ELSE 'away' END AS warsaw_location,
    AVG(home_goal) OVER(PARTITION BY season) AS season_homeavg,
    AVG(away_goal) OVER(PARTITION BY season) AS season_awayavg
FROM   match
WHERE  hometeam_id = 8673 
       OR awayteam_id = 8673
ORDER  BY (home_goal + away_goal) DESC
LIMIT 20

 * sqlite:///data/soccer.db
Done.


date,season,home_goal,away_goal,warsaw_location,season_homeavg,season_awayavg
2013-09-14 00:00:00,2013/2014,3,5,away,1.7666666666666666,1.2333333333333334
2014-09-13 00:00:00,2014/2015,4,3,home,1.5666666666666667,1.3333333333333333
2013-07-20 00:00:00,2013/2014,5,1,home,1.7666666666666666,1.2333333333333334
2013-02-23 00:00:00,2012/2013,3,2,away,1.5666666666666667,1.1333333333333333
2013-06-02 00:00:00,2012/2013,5,0,home,1.5666666666666667,1.1333333333333333
2012-10-28 00:00:00,2012/2013,3,2,home,1.5666666666666667,1.1333333333333333
2013-10-20 00:00:00,2013/2014,4,1,home,1.7666666666666666,1.2333333333333334
2013-12-15 00:00:00,2013/2014,4,1,home,1.7666666666666666,1.2333333333333334
2013-09-25 00:00:00,2013/2014,2,3,away,1.7666666666666666,1.2333333333333334
2014-08-09 00:00:00,2014/2015,5,0,home,1.5666666666666667,1.3333333333333333


## PARTITION BY multiple columns
---

The `PARTITION BY` clause can be used to break out window averages by multiple data points (columns). You can even calculate the information you want to use to partition your data! For example, you can calculate average goals scored by season and by country, or by the calendar year (taken from the date column).

In this exercise, you will calculate the average number home and away goals scored *Legia Warszawa*, and their opponents, partitioned by the month in each season.

### Instructions

Construct two window functions partitioning the average of home and away goals by season and month.
Filter the dataset by *Legia Warszawa's* team ID (`8673`) so that the window calculation only includes matches involving them.

`SELECT date,
        season,
        home_goal,
        away_goal,
 CASE WHEN hometeam_id = 8673 THEN 'home' 
     ELSE 'away' END AS warsaw_location,
     AVG(home_goal) OVER(PARTITION BY season, EXTRACT(month FROM date)) AS season_mo_home
     AVG(away_goal) OVER(PARTITION BY season, EXTRACT(month FROM date)) AS season_mo_away
 FROM   match
 WHERE  hometeam_id = 8673 
        OR awayteam_id = 8673
 ORDER  BY (home_goal + away_goal) DESC`

## Slide to the left
---

Sliding windows allow you to create running calculations between any two points in a window using functions such as `PRECEDING`, `FOLLOWING`, and `CURRENT ROW`. You can calculate running counts, sums, averages, and other aggregate functions between any two points you specify in the data set.

In this exercise, you will expand on the examples discussed in the video, calculating the running total of goals scored by the *FC Utrecht* when they were the home team during the 2011/2012 season. Do they score more goals at the end of the season as the home or away team?

### Instructions

Complete the window function by:
    
Assessing the running *total* of home goals scored by *FC Utrecht*.

Assessing the running *average* of home goals scored.

Ordering both the running average and running total by `date`.

In [14]:
%%sql

SELECT date,
       home_goal,
       away_goal,
       SUM(home_goal)
         OVER(
           ORDER BY DATE ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS
       running_total,
       AVG(home_goal)
         OVER(
           ORDER BY DATE ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS
       running_avg
FROM   match
WHERE  hometeam_id = 9908
       AND season = '2011/2012'

 * sqlite:///data/soccer.db
Done.


date,home_goal,away_goal,running_total,running_avg
2011-08-14 00:00:00,2,2,2,2.0
2011-08-27 00:00:00,3,1,5,2.5
2011-09-18 00:00:00,2,2,7,2.3333333333333335
2011-10-01 00:00:00,3,0,10,2.5
2011-10-22 00:00:00,1,4,11,2.2
2011-11-06 00:00:00,6,4,17,2.8333333333333335
2011-12-04 00:00:00,2,6,19,2.7142857142857144
2011-12-11 00:00:00,2,2,21,2.625
2012-01-22 00:00:00,1,1,22,2.4444444444444446
2012-02-12 00:00:00,1,1,23,2.3


## Slide to the right
---

Now let's see how *FC Utrecht* performs when they're the away team. You'll notice that the total for the season is at the bottom of the data set you queried. Depending on your results, this could be pretty long, and scrolling down is not very helpful.

In this exercise, you will slightly modify the query from the previous exercise by sorting the data set in reverse order and calculating a backward running total **from the** `CURRENT ROW` **to the end of the data set** (earliest record).

### Instructions

Complete the window function by:

Assessing the running *total* of home goals scored by *FC Utrecht*.

Assessing the running *average* of home goals scored.

Ordering both the running average and running total by `date`, *descending*.

In [16]:
%%sql

SELECT DATE,
       home_goal,
       away_goal,
       SUM(home_goal)
         over(
           ORDER BY DATE DESC ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING)
       AS
       running_total,
       Avg(home_goal)
         over(
           ORDER BY DATE DESC ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING)
       AS
       running_avg
FROM   match
WHERE  awayteam_id = 9908
       AND season = '2011/2012'

 * sqlite:///data/soccer.db
Done.


date,home_goal,away_goal,running_total,running_avg
2012-05-06 00:00:00,1,3,25,1.4705882352941178
2012-04-21 00:00:00,0,2,24,1.5
2012-04-12 00:00:00,3,0,24,1.6
2012-03-25 00:00:00,3,1,21,1.5
2012-03-11 00:00:00,1,1,18,1.3846153846153846
2012-02-26 00:00:00,1,0,17,1.4166666666666667
2012-02-05 00:00:00,0,2,16,1.4545454545454546
2012-01-28 00:00:00,2,0,16,1.6
2011-12-17 00:00:00,1,0,14,1.5555555555555556
2011-11-25 00:00:00,2,0,13,1.625


## Setting up the home team CTE
---
In this course, we've covered ways in which you can use `CASE` statements, subqueries, common table expressions, and window functions in your queries to structure a data set that best meets your needs. For this exercise, you will be using all of these concepts to generate a list of matches in which Manchester United was defeated during the 2014/2015 English Premier League season.

Your first task is to create the first query that filters for matches where Manchester United played as the home team. This will become a common table expression in a later exercise.

### Instructions

Create a `CASE` statement that identifies each match as a win, lose, or tie for Manchester United.

Fill out the logical operators for each `WHEN` clause in the `CASE` statement (equals, greater than, less than).

Join the tables on home team ID from `match`, and `team_api_id` from `team`.

Filter the query to only include games from the 2014/2015 season where Manchester United was the home team.

In [17]:
%%sql

SELECT m.id,
       t.team_long_name,
       CASE
         WHEN m.home_goal > m.away_goal THEN 'MU Win'
         WHEN m.home_goal < m.away_goal THEN 'MU Loss'
         ELSE 'Tie'
       END AS outcome
FROM   match AS m
       LEFT JOIN team AS t
              ON m.hometeam_id = t.team_api_id
WHERE  season = '2014/2015'
       AND t.team_long_name = 'Manchester United' 

 * sqlite:///data/soccer.db
Done.


id,team_long_name,outcome
4013,Manchester United,MU Loss
4031,Manchester United,MU Win
4051,Manchester United,MU Win
4062,Manchester United,MU Win
4085,Manchester United,MU Win
4105,Manchester United,MU Win
4145,Manchester United,MU Loss
4164,Manchester United,MU Win
4181,Manchester United,MU Win
4203,Manchester United,MU Win


## Setting up the away team CTE
---

Great job! Now that you have a query identifying the home team in a match, you will perform a similar set of steps to identify the away team. Just like the previous step, you will join the `match` and `team` tables. Each of these two queries will be declared as a Common Table Expression in the following step.

The primary difference in this query is that you will be joining the tables on `awayteam_id`, and reversing the match outcomes in the `CASE` statement.

When altering `CASE` statement logic in your own work, you can reverse either the logical condition (i.e., `home_goal > away_goal`) or the outcome in `THEN` -- just make sure you only reverse one of the two!

### Instructions

Complete the `CASE` statement syntax.

Fill out the logical operators identifying each match as a win, loss, or tie for Manchester United.

Join the table on `awayteam_id`, and `team_api_id`.

In [18]:
%%sql

SELECT m.id,
       t.team_long_name,
       CASE
         WHEN m.home_goal > m.away_goal THEN 'MU Loss'
         WHEN m.home_goal < m.away_goal THEN 'MU Win'
         ELSE 'Tie'
       END AS outcome
FROM   match AS m
       LEFT JOIN team AS t
              ON m.awayteam_id = t.team_api_id
WHERE  season = '2014/2015'
       AND t.team_long_name = 'Manchester United' 

 * sqlite:///data/soccer.db
Done.


id,team_long_name,outcome
4026,Manchester United,MU Loss
4039,Manchester United,MU Win
4075,Manchester United,MU Win
4089,Manchester United,Tie
4117,Manchester United,Tie
4126,Manchester United,Tie
4136,Manchester United,Tie
4155,Manchester United,MU Win
4178,Manchester United,Tie
4197,Manchester United,MU Loss


## Putting the CTEs together
---

Now that you've created the two subqueries identifying the `home` and `away` team opponents, it's time to rearrange your query with the home and away subqueries as Common Table Expressions (CTEs). You'll notice that the main query includes the phrase, `SELECT DISTINCT`. Without identifying only `DISTINCT` matches, you will return a duplicate record for each game played.

Continue building the query to extract all matches played by Manchester United in the 2014/2015 season.

### Instructions

Declare the `home` and `away` CTEs before your main query.

Join your CTEs to the match table using a `LEFT JOIN`.

Select the relevant data from the CTEs into the main query.

Select the date from `match`, team names from the CTEs, and home/ away goals from `match` in the main query.

In [19]:
%%sql

WITH home AS (
  SELECT m.id, t.team_long_name,
    CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
           WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id),

away AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
           WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)

SELECT DISTINCT
    m.date,
    home.team_long_name AS home_team,
    away.team_long_name AS away_team,
    m.home_goal,
    m.away_goal
FROM match AS m
LEFT  JOIN home  ON m.id = home.id
LEFT  JOIN away  ON m.id = away.id
WHERE m.season = '2014/2015'
      AND (home.team_long_name = 'Manchester United' 
           OR away.team_long_name = 'Manchester United')

 * sqlite:///data/soccer.db
Done.


date,home_team,away_team,home_goal,away_goal
2014-08-16 00:00:00,Manchester United,Swansea City,1,2
2014-11-02 00:00:00,Manchester City,Manchester United,1,0
2014-11-08 00:00:00,Manchester United,Crystal Palace,1,0
2014-11-22 00:00:00,Arsenal,Manchester United,1,2
2014-11-29 00:00:00,Manchester United,Hull City,3,0
2014-12-02 00:00:00,Manchester United,Stoke City,2,1
2014-12-08 00:00:00,Southampton,Manchester United,1,2
2014-12-14 00:00:00,Manchester United,Liverpool,3,0
2014-12-20 00:00:00,Aston Villa,Manchester United,1,1
2014-12-26 00:00:00,Manchester United,Newcastle United,3,1


## Add a window function
---

Fantastic! You now have a result set that retrieves the match date, home team, away team, and the goals scored by each team. You have one final component of the question left -- how badly did Manchester United lose in each match?

In order to determine this, let's add a window function to the main query that ranks matches by the absolute value of the difference between `home_goal` and `away_goal`. This allows us to directly compare the difference in scores without having to consider whether Manchester United played as the home or away team!

The equation is complete for you -- all you need to do is properly complete the window function!

### Instructions

Set up the CTEs so that the home and away teams each have a name, ID, and score associated with them.

Select the date, home team name, away team name, home goal, and away goals scored in the main query.

Rank the matches and order by the difference in scores in descending order.

In [20]:
%%sql

WITH home AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
           WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id),

away AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
           WHEN m.home_goal < m.away_goal THEN 'MU Win' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)

SELECT DISTINCT
    m.date,
    home.team_long_name AS home_team,
    away.team_long_name AS away_team,
    m.home_goal, m.away_goal,
    RANK() OVER(ORDER BY ABS(home_goal - away_goal) DESC) as match_rank

FROM match AS m
LEFT JOIN home ON m.id = home.id
LEFT JOIN away ON m.id = away.id
WHERE m.season = '2014/2015'
      AND ((home.team_long_name = 'Manchester United' AND home.outcome = 'MU Loss')
      OR (away.team_long_name = 'Manchester United' AND away.outcome = 'MU Loss'))

 * sqlite:///data/soccer.db
Done.


date,home_team,away_team,home_goal,away_goal,match_rank
2015-04-26 00:00:00,Everton,Manchester United,3,0,1
2014-09-21 00:00:00,Leicester City,Manchester United,5,3,2
2014-08-16 00:00:00,Manchester United,Swansea City,1,2,3
2014-11-02 00:00:00,Manchester City,Manchester United,1,0,3
2015-01-11 00:00:00,Manchester United,Southampton,0,1,3
2015-02-21 00:00:00,Swansea City,Manchester United,2,1,3
2015-04-18 00:00:00,Chelsea,Manchester United,1,0,3
2015-05-02 00:00:00,Manchester United,West Bromwich Albion,0,1,3
